In [132]:
import pandas as pd

In [133]:
df = pd.read_csv('full_df.csv')

In [134]:
df['departure_date'] = pd.to_datetime(df['departure_date'])
df = df.dropna(subset=['load_capacity_pounds', 'fuel_type'])

In [135]:
def classify_route(distance):
    if distance <= 1000:
        return 'Short'
    
    else: 
        return 'Long'
df['route_category'] = df['distance'].apply(classify_route)
schedule = df.sort_values(by = ['truck_id', 'departure_date'])
schedule.head()

,Unnamed: 0.1,Unnamed: 0,Index,truck_id,route_id,departure_date,estimated_arrival,delay,truck_age,load_capacity_pounds,...,gender,age,experience,driving_style,ratings,average_speed_mph,average_actual_hours,fuel_consumed,Column1,route_category
9987,9987,10862,573.0,10008392.0,R-35dca729,2019-01-01,15:36.0,1.0,7.0,10000.0,...,male,56.0,18.0,proactive,6.0,60.74,41.372901,109.260435,10000,Long
10111,10111,10990,2711.0,10008392.0,R-7ab5ae85,2019-01-09,22:12.0,0.0,7.0,10000.0,...,male,56.0,18.0,proactive,6.0,60.74,32.408956,85.587826,10000,Long
10416,10416,11308,5108.0,10008392.0,R-51d1a32a,2019-01-17,57:36.0,1.0,7.0,10000.0,...,male,56.0,18.0,proactive,6.0,60.74,48.531939,128.166522,10000,Long
10540,10540,11439,7253.0,10008392.0,R-e6950f9c,2019-01-25,36:36.0,0.0,7.0,10000.0,...,male,56.0,18.0,proactive,6.0,60.74,35.078696,92.638261,10000,Long
10546,10546,11445,9231.0,10008392.0,R-06dfe39e,2019-02-02,55:12.0,1.0,7.0,10000.0,...,male,56.0,18.0,proactive,6.0,60.74,49.322522,130.254348,10000,Long


In [136]:
truck_distribution = df.groupby(['route_category','fuel_type'])['truck_id'].nunique().reset_index()
truck_distribution

,route_category,fuel_type,truck_id
0,Long,diesel,582
1,Long,gas,486
2,Short,diesel,313
3,Short,gas,255


In [137]:
df.drop(columns = ['Unnamed: 0.1','Unnamed: 0'], inplace = True)

In [138]:
import pandas as pd

truck_totals = {}

for index, row in df.iterrows():
    truck_id = row['truck_id']
    if truck_id not in truck_totals:
        truck_totals[truck_id] = {
            'total_distance': 0,
            'capacity': row['load_capacity_pounds'],
            'routes': [],
            'fuel_type': row['fuel_type'],
            'mileage_mpg': row['mileage_mpg'],
            'average_hours': row['average_hours'],
            'average_speed_mph': row['average_speed_mph']
        }
    truck_totals[truck_id]['total_distance'] += row['distance']
    truck_totals[truck_id]['routes'].append({
        'route_id': row['route_id'],
        'distance': row['distance'],
        'departure_date': row['departure_date'],
        'route_category': row['route_category'],
        'mileage_mpg': row['mileage_mpg'],
        'average_hours': row['average_hours'],
        'average_speed_mph': row['average_speed_mph']
    })

# fuel consumption for each truck
def calculate_fuel_consumption(truck_data):
    total_fuel_consumption = 0
    for route in truck_data['routes']:
        # fuel consumption for each route
        total_fuel_consumption += route['distance'] / truck_data['mileage_mpg']
    return total_fuel_consumption

# possible diesel-gas truck swaps
swap_candidates = []
for diesel_truck, diesel_data in truck_totals.items():
    if diesel_data['fuel_type'] != 'diesel':
        continue
    for gas_truck, gas_data in truck_totals.items():
        if gas_data['fuel_type'] != 'gas' or diesel_data['capacity'] != gas_data['capacity']:
            continue
        
        # fuel consumption before and after swap
        original_fuel_diesel = calculate_fuel_consumption(diesel_data)
        original_fuel_gas = calculate_fuel_consumption(gas_data)
        
       
        new_fuel_diesel = 0
        new_fuel_gas = 0
        
        for route in diesel_data['routes']:
            if route['route_category'] == 'long':  # Diesel truck will take long route
                new_fuel_diesel += route['distance'] / diesel_data['mileage_mpg']
            else:  # Gas truck will take short route
                new_fuel_gas += route['distance'] / gas_data['mileage_mpg']
            
       
        for route in gas_data['routes']:
            if route['route_category'] == 'short':  # Diesel truck will take long route
                new_fuel_gas += route['distance'] / gas_data['mileage_mpg']
            else:  # Gas truck will take short route
                new_fuel_diesel += route['distance'] / diesel_data['mileage_mpg']
        
        # Calculate the fuel savings
        fuel_savings = (original_fuel_diesel + original_fuel_gas) - (new_fuel_diesel + new_fuel_gas)
        
        if fuel_savings > 0:
            swap_candidates.append((diesel_truck, gas_truck, fuel_savings))

# Step 4: Sort swaps by the highest fuel savings
swap_candidates.sort(key=lambda x: x[2], reverse=True)

# Step 5: Reassign trucks with the largest fuel savings
reassignments = []
assigned_trucks = set()

for diesel_truck, gas_truck, _ in swap_candidates:
    if diesel_truck in assigned_trucks or gas_truck in assigned_trucks:
        continue
    reassignments.append((diesel_truck, gas_truck))
    assigned_trucks.add(diesel_truck)
    assigned_trucks.add(gas_truck)

# Step 6: Generate new schedules for reassigned trucks
new_schedules = []
reassigned_trucks = set()

for diesel_truck, gas_truck in reassignments:
    reassigned_trucks.update([diesel_truck, gas_truck])

    # Reassign routes from diesel to gas truck
    for route in truck_totals[diesel_truck]['routes']:
        new_schedules.append({
            'truck_id': gas_truck,
            'route_id': route['route_id'],
            'fuel_type': truck_totals[gas_truck]['fuel_type'],
            'distance': route['distance'],
            'departure_date': route['departure_date'],
            'load_capacity': truck_totals[gas_truck]['capacity'],
            'route_category': route['route_category'],
            'mileage_mpg': truck_totals[gas_truck]['mileage_mpg'],
            'average_hours': truck_totals[gas_truck]['average_hours'],
            'average_speed_mph': truck_totals[gas_truck]['average_speed_mph'],
            'reassigned': True
        })
    
    # Reassign routes from gas to diesel truck
    for route in truck_totals[gas_truck]['routes']:
        new_schedules.append({
            'truck_id': diesel_truck,
            'route_id': route['route_id'],
            'fuel_type': truck_totals[diesel_truck]['fuel_type'],
            'distance': route['distance'],
            'departure_date': route['departure_date'],
            'load_capacity': truck_totals[diesel_truck]['capacity'],
            'route_category': route['route_category'],
            'mileage_mpg': truck_totals[diesel_truck]['mileage_mpg'],
            'average_hours': truck_totals[diesel_truck]['average_hours'],
            'average_speed_mph': truck_totals[diesel_truck]['average_speed_mph'],
            'reassigned': True
        })

# Step 7: Retain original schedules for unassigned trucks
for index, row in df.iterrows():
    if row['truck_id'] not in reassigned_trucks:
        new_schedules.append({
            'truck_id': row['truck_id'],
            'route_id': row['route_id'],
            'fuel_type': row['fuel_type'],
            'distance': row['distance'],
            'departure_date': row['departure_date'],
            'load_capacity': row['load_capacity_pounds'],
            'route_category': row['route_category'],
            'mileage_mpg': row['mileage_mpg'],
            'average_hours': row['average_hours'],
            'average_speed_mph': row['average_speed_mph'],
            'reassigned': False
        })

# Convert to DataFrame and save
full_schedule_df = pd.DataFrame(new_schedules)
#full_schedule_df.to_csv('checking_code.csv', index=False)
print("Optimized truck schedules saved to optimized_truck_schedules.csv.")


Optimized truck schedules saved to optimized_truck_schedules.csv.


In [143]:
original_fuel_used = (df['distance'] / df['mileage_mpg']).sum()
new_fuel_used = (full_schedule_df['distance'] / full_schedule_df['mileage_mpg']).sum()


In [144]:
print(f"Original fuel consumption: {original_fuel_used:.2f}")
print(f"Optimized fuel consumption: {new_fuel_used:.2f}")


Original fuel consumption: 703306.78
Optimized fuel consumption: 665346.26
